<a href="https://colab.research.google.com/github/mtpradoc/BigQueryAPI/blob/main/06_JOINING_DATA_Dataset_Stack_Overflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Programmatically connect to BigQuery

In order to use a public dataset in BigQuery we need to programmatically authenticate to the google cloud platform

##1. Authenticate to GCP

In [ ]:
from google.colab import auth
auth.authenticate_user()

Let's specify which project_id we are going to use. It can be any

In [ ]:
project_id = 'glossy-mason-326213'

##2. Connect to the BigQuery API

In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client(project=project_id)

##3. Access the Dataset (public or private)

Let's put a reference for the dataset and project where the dataset we are going to work with

In [ ]:
serverdb = "bigquery-public-data"
db = "stackoverflow"

In [ ]:
dataset_ref = client.dataset(db, project=serverdb)

dataset = client.get_dataset(dataset_ref)

##4. List your tables

In [ ]:
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables]

print(list_of_tables)

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


##5. Check table 1

In [ ]:
answers_table_ref = dataset_ref.table("posts_answers")

answers_table = client.get_table(answers_table_ref)

df = client.list_rows(answers_table, max_results=500).to_dataframe()
df

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,68472973,None,<p>Thank you. After thinking through your ans...,None,None,0,None,2021-07-21 16:09:49.560000+00:00,None,2021-07-21 16:09:49.560000+00:00,None,None,None,None,11880888.0,68461807,2,0,None,None
1,68472974,None,<pre><code>SELECT age((((m.message -&gt; 'head...,None,None,0,None,2021-07-21 16:09:54.130000+00:00,None,2021-07-21 16:09:54.130000+00:00,None,None,None,None,14132876.0,68471992,2,0,None,None
2,68472975,None,<p>I believe <code>Identifiable</code> is a pr...,None,None,1,None,2021-07-21 16:09:56.317000+00:00,None,2021-07-21 16:09:56.317000+00:00,None,None,None,None,16487083.0,68472232,2,0,None,None
3,68472976,None,<p>Funny enough I had the same problem with th...,None,None,0,None,2021-07-21 16:10:06.223000+00:00,None,2021-07-21 16:10:06.223000+00:00,None,None,None,None,9497800.0,64728953,2,0,None,None
4,68472989,None,<p>You either need to set a <code>height</code...,None,None,0,None,2021-07-21 16:10:45.250000+00:00,None,2021-07-21 16:10:45.250000+00:00,None,None,None,None,6035993.0,68472903,2,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,68475650,None,<p>To get your date you need to Format it</p>\...,None,None,2,None,2021-07-21 19:48:02.350000+00:00,None,2021-07-21 19:48:02.350000+00:00,None,None,None,None,5193536.0,68475585,2,0,None,None
496,68475660,None,<p>This is what you are looking for:</p>\n<pre...,None,None,3,None,2021-07-21 19:48:36.797000+00:00,None,2021-07-21 19:48:36.797000+00:00,None,None,None,None,16424558.0,68475255,2,0,None,None
497,68475662,None,"<p>I got the same issue, and after debug it i ...",None,None,0,None,2021-07-21 19:48:41.243000+00:00,None,2021-07-21 19:48:41.243000+00:00,None,None,None,None,1163531.0,67583613,2,0,None,None
498,68475663,None,<p>You should set a specific ticker locator to...,None,None,0,None,2021-07-21 19:48:50.107000+00:00,None,2021-07-21 19:48:50.107000+00:00,None,None,None,None,3316560.0,68475195,2,0,None,None


##5. Check table 2

In [ ]:
questions_table_ref = dataset_ref.table("posts_questions")

questions_table = client.get_table(questions_table_ref)

client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,68760184,SpawnPrefabAtGeoLocation with mapbox-streets-v8?,<p>How can we spawn custom prefabs at geo loca...,None,0,0,None,2021-08-12 15:22:56.787000+00:00,None,2021-08-12 15:22:56.787000+00:00,None,None,None,None,8169506,None,1,0,mapbox-unity,2
1,68776566,Why to use process.nextTick() to call Writeabl...,"<p>I was reading the <a href=""https://nodejs.o...",None,0,0,None,2021-08-13 17:56:46.653000+00:00,None,2021-08-13 17:56:46.653000+00:00,None,None,None,None,11255630,None,1,0,javascript|node.js|stream|buffer,2
2,68847727,Share button in browser copy old address url,<p>I dont know but my share button in browser ...,None,0,0,None,2021-08-19 12:16:22.210000+00:00,None,2021-08-19 12:16:22.210000+00:00,None,None,None,None,15916653,None,1,0,url|mobile,2
3,68863312,Changed a Virtual Switch on a node and destroy...,<p>I have a Windows Failover cluster with 2 'b...,None,0,0,None,2021-08-20 13:46:29.450000+00:00,None,2021-08-20 13:46:29.450000+00:00,None,None,None,None,3533172,None,1,0,failovercluster,2
4,69023598,Problem about exporting data by the function “...,<p>1.The field type defined in the database is...,None,0,0,None,2021-09-02 03:00:15+00:00,None,2021-09-02 03:00:15+00:00,None,None,None,None,15504166,None,1,0,dolphindb,2


##6. Explore your data

Something to have into account:

**posts_answers** has a column called **parent_id** which identifies the ID of the question each answer is responding to. **posts_answers** also has an **owner_user_id** column which specifies the ID of the user who answered the question.

 Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [ ]:
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config)

questions_results = questions_query_job.to_dataframe()

print(questions_results.head())

         id                                              title  owner_user_id
0  68739751  where can I find the exact error[] collection ...     12701829.0
1  68936196  BigQuery Server Location Comparison: Is EU loc...     12343115.0
2  69060625                         Array Filter on DataStudio     16709866.0
3  68758480     Bigquery DR complaint and Multi region support     16650439.0
4  69024811  How can I export my openrtb proto data to bigq...     16811680.0


Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.


In [ ]:
answers_query = """
                SELECT pa.id, pa.body, pa.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS pq
                  INNER JOIN
                  `bigquery-public-data.stackoverflow.posts_answers` AS pa
                  ON pa.parent_id=pq.id
                WHERE pq.tags LIKE '%bigquery%'
                """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=26*(1024**3))
answers_query_job = client.query(answers_query, job_config=safe_config)

answers_results = answers_query_job.to_dataframe()

print(answers_results.head())
                

You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:

user_id - contains the owner_user_id column from the posts_answers table
number_of_answers - contains the number of answers the user has written to "bigquery"-related questions

In [ ]:
bigquery_experts_query = """
                         SELECT pa.owner_user_id AS user_id,
                                COUNT(1) AS number_of_answers
                         FROM
                                `bigquery-public-data.stackoverflow.posts_answers` AS pa
                                INNER JOIN
                                `bigquery-public-data.stackoverflow.posts_questions` AS pq
                                ON pa.parent_id=pq.id
                         WHERE pq.tags LIKE '%bigquery%'
                         GROUP BY user_id
                         """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

print(bigquery_experts_results.head())

     user_id  number_of_answers
0   222901.0                  3
1  9593895.0                  1
2  2067753.0                  6
3  1329634.0                  1
4  9713409.0                  1


#7.Build a General Python Function

Convert what you've done to a general function a website could call on the backend to get experts on any topic

In [ ]:
def expert_finder(topic,client):
    '''
    Returns a Dataframe with the user IDs who have written Stack Overflow answers on a topic.
    
    Inputs:
      topic: A string with the topic of interest
      client: A Client object that specified the connection to the Stack Overflow dataset
    
    Outputs:
      results: A Dataframe with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts results shown above.
    '''
    my_query = """
               SELECT pa.owner_user_id AS user_id,
                      COUNT(1) AS number_of_answers
               FROM
                      `bigquery-public-data.stackoverflow.posts_questions` AS pq
                      INNER JOIN
                      `bigquery-public-data.stackoverflow.posts_answers` AS pa
                      ON pq.id = pa.parent_id
               WHERE pq.tags LIKE '%"""+topic+"""%'
               GROUP BY pa.owner_user_id
               """
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    my_query_job = client.query(my_query,job_config=safe_config)

    results = my_query_job.to_dataframe()

    return results

In [ ]:
expert_finder("bigquery", client)

,user_id,number_of_answers
0,6460656.0,8
1,14970137.0,1
2,6786879.0,1
3,3652345.0,7
4,6158051.0,1
...,...,...
5076,11821225.0,1
5077,5609444.0,1
5078,6390170.0,1
5079,11416350.0,1
